In [51]:
import importlib
from pybit.unified_trading import HTTP
import os
from utils import utils
importlib.reload(utils)

class BybitWrapper():

    def __init__(self, demo=False, api_key=None, api_secret=None):
        print(f'Wrapper Activated. Demo Mode == {demo}')
        self.demo = demo
        
        if self.demo:   
            self.api_key = api_key or os.getenv("BYBIT_API_KEY_TEST")
            self.api_secret = api_secret or os.getenv("BYBIT_API_SECRET_TEST")
            self.session = HTTP(api_key=self.api_key, api_secret=self.api_secret, demo=demo, log_requests=True)
        else:
            self.api_key = api_key or os.getenv("BYBIT_API_KEY")
            self.api_secret = api_secret or os.getenv("BYBIT_API_SECRET")
            self.session = HTTP(api_key=self.api_key, api_secret=self.api_secret, demo=demo, log_requests=True)
        

    # Account Data Endpoints 

    def transaction_log(self, account_type='UNIFIED', market=None, coin=None):
        
        response = self.session.get_transaction_log(accountType=account_type,category=market,currency=coin)

        return utils.parse_transaction_log(response)


    def wallet_balance(self, account_type: str, coin: str):
        
        response = self.session.get_wallet_balance(accountType=account_type, coin=coin)
        
        return utils.parse_wallet_balance(response)
    
    def get_coin_balance(self, account_type: str = 'UNIFIED', coin: str = None, member_id: str = None, with_bonus: int = 0):
        
        if self.demo:
            raise RuntimeError("This operation is not allowed in demo mode.")
        else:
            response = self.session.get_coins_balance(
                accountType=account_type,
                coin=coin,
                memberId=member_id,
                withBonus=with_bonus
            )
            return utils.parse_coin_balance(response=response)
    
    def get_api_details(self):
        if self.demo:
            raise RuntimeError("This operation is not allowed in demo mode.")
        else:
            response=self.session.get_api_key_information()
            return response

    # Market Data Endpoints (Common for Spot and Perpetual)
    def get_orderbook(self, ticker: str, category: str, limit: int = 100):
        response=self.session.get_orderbook(category=category, symbol=ticker, limit=limit)
        return utils.parse_orderbook(response=response)
    
    def get_candles(self, market, ticker, interval: str = "60", limit: int = 10):
        response=self.session.get_kline(category=market, symbol=ticker, interval=interval, limit=limit)
        return utils.parse_klines(response)


    # Spot Market Endpoints
    def place_spot_order(self, symbol: str, side: str, order_type: str, qty: float, price: float = None):
        pass
    
    def cancel_spot_order(self, symbol: str, order_id: str):
        pass

    def spot_order_history(self, market: str = 'spot', ticker='BTCUSDT', limit: int = 100):
        
        
        response = self.session.get_order_history(category=market, 
                                                  symbol=ticker,
                                                  limit=limit
                                                  )
        
        
        return utils.parse_order_history(response)

    # Perpetual Market Endpoints

    def get_positions(self, market: str, ticker: str, settleCoin: str = None, limit: int = 20, cursor: str = None):
        response = self.session.get_positions(
            category=market,
            symbol=ticker,
            settleCoin=settleCoin,
            limit=limit,
            cursor=cursor
        )

        return utils.parse_positions(response)

    def place_perp_order(self, symbol: str, side: str, order_type: str, qty: float, price: float = None, reduce_only: bool = False):
        pass
    
    def cancel_perp_order(self, symbol: str, order_id: str):
        pass
    
    def get_perp_balance(self, coin: str = "USDT"):
        pass
    
    def get_perp_positions(self, symbol: str):
        pass
    
    def get_perp_order_history(self, symbol: str, start_time: int = None, end_time: int = None, limit: int = 50):
        pass

    # Wallet & Transfer Endpoints (Common)
    def get_wallet_balance(self, coin: str = "USDT"):
        pass

    def transfer_funds(self, coin: str, amount: float, from_account: str, to_account: str):
        pass

    def get_deposit_history(self, coin: str = "USDT"):
        pass

    def get_withdrawal_history(self, coin: str = "USDT"):
        pass 

In [52]:
wrapper=BybitWrapper(demo=False)
test_wrapper=BybitWrapper(demo=True)

Wrapper Activated. Demo Mode == False
Wrapper Activated. Demo Mode == True


In [49]:
wrapper.get_coin_balance() # not available in demo mode 

,account_type,member_id,timestamp,coin,wallet_balance,transfer_balance,bonus
0,UNIFIED,149492798,2024-08-11 22:02:09,BTC,0.000170,0.000170,None
1,UNIFIED,149492798,2024-08-11 22:02:09,MODE,0.009878,0.009878,None
2,UNIFIED,149492798,2024-08-11 22:02:09,USDT,866.974900,866.974900,None
3,UNIFIED,149492798,2024-08-11 22:02:09,ONDO,990.948060,990.948060,None
4,UNIFIED,149492798,2024-08-11 22:02:09,USDE,0.002568,0.002568,None


In [56]:
test_wrapper.wallet_balance(account_type='UNIFIED', coin='BTC,USDT,ONDO') 

,account_type,total_equity,total_wallet_balance,total_margin_balance,total_available_balance,coin,equity,usd_value,wallet_balance,free,locked,spot_hedging_qty,borrow_amount,available_to_withdraw,accrued_interest,unrealised_pnl,cum_realised_pnl,margin_collateral,collateral_switch
0,UNIFIED,161218.36484,118698.749691,118869.745559,109629.575248,BTC,0.670460,39756.472771,0.670460,0.0,0.0,0.0,0.0,0.670460,0.0,0.00000,-0.000171,True,False
1,UNIFIED,161218.36484,118698.749691,118869.745559,109629.575248,USDT,68834.810558,68858.145559,68663.872638,0.0,0.0,0.0,0.0,68636.232158,0.0,170.93792,-1016.477362,True,True
2,UNIFIED,161218.36484,118698.749691,118869.745559,109629.575248,ONDO,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,True,False


In [42]:
test_wrapper.wallet_balance(account_type='UNIFIED', coin='BTC,ETH,USDT')

,account_type,total_equity,total_wallet_balance,total_margin_balance,total_available_balance,coin,equity,usd_value,wallet_balance,free,locked,spot_hedging_qty,borrow_amount,available_to_withdraw,accrued_interest,unrealised_pnl,cum_realised_pnl,margin_collateral,collateral_switch
0,UNIFIED,160861.665455,118657.752596,118734.104142,109496.926631,BTC,0.670460,39548.792895,0.670460,0.0,0.0,0.0,0.0,0.670460,0.0,0.0000,-0.000171,True,False
1,UNIFIED,160861.665455,118657.752596,118734.104142,109496.926631,ETH,1.000000,2578.768418,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0000,0.000000,True,False
2,UNIFIED,160861.665455,118657.752596,118734.104142,109496.926631,USDT,68740.223038,68741.254142,68663.872638,0.0,0.0,0.0,0.0,68650.146238,0.0,76.3504,-1016.477362,True,True


In [43]:
wrapper.transaction_log(market='spot') #spot, linear 

,id,symbol,category,side,transaction_time,type,qty,size,currency,trade_price,funding,fee,cash_flow,change,cash_balance,fee_rate,bonus_change,trade_id,order_id,order_link_id
0,149492798_22009_37184356008_1,BTCUSDT,spot,Buy,2024-08-11 20:58:56,TRADE,0.0001,0.0,BTC,58541.43,0.0,1.700000e-07,0.000170,0.000170,0.000170,0.001,0.0,2290000000308574873,1750483853426127104,
1,149492798_22009_37184356008_0,BTCUSDT,spot,Buy,2024-08-11 20:58:56,TRADE,-9.9520,0.0,USDT,58541.43,0.0,0.000000e+00,-9.952043,-9.952043,866.974984,0.000,0.0,2290000000308574873,1750483853426127104,


In [44]:
test_wrapper.wallet_balance(account_type='UNIFIED', coin='BTC')


,account_type,total_equity,total_wallet_balance,total_margin_balance,total_available_balance,coin,equity,usd_value,wallet_balance,free,locked,spot_hedging_qty,borrow_amount,available_to_withdraw,accrued_interest,unrealised_pnl,cum_realised_pnl,margin_collateral,collateral_switch
0,UNIFIED,160861.665455,118657.752596,118734.104142,109496.926631,BTC,0.67046,39548.792895,0.67046,0.0,0.0,0.0,0.0,0.67046,0.0,0.0,-0.000171,True,False


In [45]:
test_wrapper.spot_order_history(ticker='BTCUSDT')


,order_id,order_link_id,symbol,price,quantity,side,position_idx,order_status,cancel_type,reject_reason,...,cum_exec_fee,time_in_force,order_type,trigger_price,take_profit,stop_loss,reduce_only,close_on_trigger,created_time,updated_time
0,1750492494363364608,1723410966426,BTCUSDT,0.00,10000.000000,Buy,0,Filled,UNKNOWN,EC_NoError,...,0.000171,IOC,Market,0.0,0.0,0.0,False,False,2024-08-11 21:16:06,2024-08-11 21:16:06
1,1750492296157334784,1723410942789,BTCUSDT,58551.46,0.295244,Buy,0,Cancelled,CancelByUser,EC_PerCancelRequest,...,0.000000,GTC,Limit,0.0,0.0,0.0,False,False,2024-08-11 21:15:42,2024-08-11 21:15:49
2,1750457736535215360,1723406822584,BTCUSDT,0.00,0.500000,Sell,0,Filled,UNKNOWN,EC_NoError,...,29.680350,IOC,Market,0.0,0.0,0.0,False,False,2024-08-11 20:07:03,2024-08-11 20:07:03


In [46]:
test_wrapper.get_positions(market='linear', ticker='BTCUSDT')

,position_idx,symbol,side,size,avg_price,position_value,unrealised_pnl,leverage,liq_price,mark_price,created_time,updated_time,position_status,trade_mode,position_balance,take_profit,stop_loss
0,0,BTCUSDT,Sell,0.046,58666.1,2698.6406,-13.7264,1.4,,58964.5,2024-08-11 18:47:02,2024-08-11 20:43:14,Normal,0,0.0,None,None


In [57]:
test_wrapper.get_positions(market='linear', ticker='ETHUSDT')

,position_idx,symbol,side,size,avg_price,position_value,unrealised_pnl,leverage,liq_price,mark_price,created_time,updated_time,position_status,trade_mode,position_balance,take_profit,stop_loss
0,0,ETHUSDT,Buy,8.53,2567.01,21896.5953,78.8172,3.0,,2576.25,2024-08-11 20:43:59,2024-08-11 20:45:57,Normal,0,0.0,None,None


In [48]:
test_wrapper.get_candles(ticker='BTCUSDT', market='linear')

,open,high,low,close,volume,turnover
t,,,,,,
2024-08-11 13:00:00,60459.1,60586.6,60134.3,60419.1,3753.557,2.265064e+08
2024-08-11 14:00:00,60419.1,60744.4,60151.6,60312.0,3554.566,2.147930e+08
2024-08-11 15:00:00,60312.0,60400.0,59908.3,60099.8,5529.911,3.326004e+08
2024-08-11 16:00:00,60099.8,60259.1,59945.1,60148.5,2276.470,1.368482e+08
2024-08-11 17:00:00,60148.5,60328.7,59989.2,60240.5,1833.467,1.102670e+08
2024-08-11 18:00:00,60240.5,60320.9,60079.1,60233.2,1186.456,7.140826e+07
2024-08-11 19:00:00,60233.2,60244.0,59508.8,59596.0,6394.032,3.819852e+08
2024-08-11 20:00:00,59596.0,59653.6,58264.0,58478.9,20426.254,1.202298e+09
2024-08-11 21:00:00,58478.9,59049.9,58288.6,58993.1,6491.286,3.807411e+08


In [ ]:
wrapper.get_api_details()